<a href="https://colab.research.google.com/github/cao-nv/visual_transformer/blob/main/transformer_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install einops

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8MB 10.7MB/s 
     |████████████████████████████████| 133kB 39.1MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 163kB 34.5MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=4bbef2607dc880cb3243892801f97d19ea331240c992403ca64b3f603afc2ff6
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=df9f2e29c6a8821a5d3c4c692dcc1537dd3760d7b112f9c1a2cdd0bbfbc69c33
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from einops import rearrange 

import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
import pdb

In [ ]:
class Residual(nn.Module):
  def __init__(self, fn): 
    super().__init__()
    self.fn = fn 

  def forward(self, x, **kwargs):
    return self.fn(x, **kwargs) + x

In [ ]:
class PreNorm(nn.Module): 
  def __init__(self, dim, fn):
    super().__init__()
    self.norm = nn.LayerNorm(dim)
    self.fn = fn 

  def forward(self, x, **kwargs):
    return self.fn(self.norm(x), **kwargs)

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, dim, hidden_dim, dropout=0.):
    super().__init__() 
    self.net = nn.Sequential(
        nn.Linear(dim, hidden_dim), 
        nn.GELU(), 
        nn.Dropout(dropout),
        nn.Linear(hidden_dim, dim), 
        nn.Dropout(dropout)
    )

  def forward(self, x): 
    return self.net(x)

In [ ]:
class Attention(nn.Module): 
  def __init__(self, dim, heads=8, dropout=0.):
    super().__init__()
    self.heads = heads 
    self.scales = heads ** (-0.5)

    self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
    self.to_out = nn.Sequential(
        nn.Linear(dim, dim), 
        nn.Dropout(dropout)
    )

  def forward(self, x, mask=None):
    b, n, _, h = *x.shape, self.heads
    qkv = self.to_qkv(x).chunk(3, dim=-1)
    q, k, v = map(lambda t: rearrange(t, "b n (h d) -> b h n d", h=h), qkv)

    dots = torch.einsum("bhid, bhjd->bhij", q, k) * self.scales

    if mask is not None: 
      mask = F.pad(mask.flatten(1), (1, 0), value=True)
      assert mask.shape[-1] == dots.shape[-1], "Mask has incorrect dimensions"
      mask = mask[:, None, :] * mask[:, :, None]
      dots.mask_fill_(~mask, float('-inf'))
      del mask 

    attn = dots.softmax(dim=-1)
    out = torch.einsum("bhij,bhjd->bhid", attn, v)
    out = rearrange(out, "b h n d -> b n (h d)")
    out = self.to_out(out)
    return out

In [ ]:
class Transformer(nn.Module):
  def __init__(self, dim, depth, heads, mlp_dim, dropout):
    super().__init__()
    self.layers = nn.ModuleList([])

    for _ in range(depth):
      self.layers.append(nn.ModuleList([
                                        Residual(PreNorm(dim, Attention(dim, heads=heads, dropout=dropout))), 
                                        Residual(PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout)))
      ]))

  def forward(self, x, mask=None):
    for attn, ff in self.layers:
      x = attn(x, mask=mask)
      x = ff(x)
    return x

In [ ]:
MIN_NUM_PATCHES = 16
class ViT(nn.Module): 
  def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3, dropout=0., emb_dropout=0.):
    super().__init__()
    assert image_size % patch_size == 0, "image size must be divisible for the patch size"
    num_patches = (image_size // patch_size) ** 2
    patch_dim = channels * patch_size**2 
    assert num_patches >= MIN_NUM_PATCHES,  f'your number of patches ({num_patches}) is way too small for attention to be effective. try decreasing your patch size'

    self.patch_size = patch_size 

    self.pos_embedding = nn.Parameter(torch.randn(1, num_patches+1, dim))
    self.patch_to_embedding = nn.Linear(patch_dim, dim, bias=False)
    self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
    self.dropout = nn.Dropout(emb_dropout)

    self.transformer = Transformer(dim, depth, heads, mlp_dim, dropout)

    self.to_cls_token = nn.Identity()

    self.mlp_head = nn.Sequential(
        nn.LayerNorm(dim), 
        nn.Linear(dim, mlp_dim), 
        nn.GELU(), 
        nn.Dropout(dropout), 
        nn.Linear(mlp_dim, num_classes)
    )

  def forward(self, img, mask=None):
    p = self.patch_size
    x = rearrange(img, "b c (h p1) (w p2) -> b (h w) (p1 p2 c)", p1=p, p2=p)
    x = self.patch_to_embedding(x)
    b, n, _ = x.shape

    cls_tokens = self.cls_token.expand(b, -1, -1)
    x = torch.cat((cls_tokens, x), dim=1)
    x += self.pos_embedding[:, :(n+1)]
    x = self.dropout(x)

    x = self.transformer(x, mask)
    x = self.to_cls_token(x[:, 0])

    return self.mlp_head(x)

In [ ]:
import os

import torch.optim as optim 
import torch.backends.cudnn as cudnn 
import torchvision 
import torchvision.transforms as transforms
import pandas as pd
import csv
import os

In [ ]:
import albumentations

In [ ]:
import wandb
wandb.init(project="ViT-cifar10", entity="caonv")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
lr = 1e-4 
bs = 64
n_epochs = 100
patch = 2


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0
start_epoch = 0 

In [ ]:
transform_train = transforms.Compose([
                                     transforms.RandomCrop(32, padding=4),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = ViT(
    image_size = 32,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 512,
    dropout = 0.1,
    emb_dropout = 0.1
)
wandb.watch(net)
net = net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True, min_lr=1e-3*1e-5, factor=0.1)

In [ ]:
def train(epoch):
  print('\nEpoch: %d' % epoch)
  net.train()
  train_loss = 0
  correct = 0
  total = 0
  
  progress_bar = notebook.tqdm(enumerate(trainloader), 
                              desc='Epoch {}'.format(epoch))  

  for batch_idx, (inputs, targets) in enumerate(trainloader):
    
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = outputs.max(1)
    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()

    progress_bar.set_description("Loss: {:.4f}, Acc: {:.4f}".format(train_loss/(batch_idx+1), 100.*correct/total, correct, total))

  return train_loss/(batch_idx+1)

In [ ]:
import time
def test(epoch):
  global best_acc
  net.eval() 
  test_loss = 0 
  correct = 0 
  total = 0 
  progress_bar = notebook.tqdm(enumerate(testloader), 
                               desc="Epoch {}".format(epoch))
  with torch.no_grad():
    for batch_idx, (inputs, targets) in progress_bar: 
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = net(inputs)
      loss = criterion(outputs, targets)
      test_loss += loss.item() 
      _, predicted = outputs.max(1)
      total += targets.size(0) 
      correct += predicted.eq(targets).sum().item()

      progress_bar.set_description("Loss: {:.4f}, Acc: {:.4f}".format(test_loss/(batch_idx+1), 100.*correct/total, correct, total))

  scheduler.step(test_loss)
  acc = 100. * correct/total

  if acc > best_acc: 
    print("Best accuracy: {}".format(acc))
  wandb.log({"test_loss": test_loss/(batch_idx+1)})
  return test_loss, acc

In [ ]:

list_loss = []
list_acc = []
for epoch in range(n_epochs):
  train(epoch)
  val_loss, val_acc = test(epoch)

  list_loss.append(val_loss)
  list_acc.append(val_acc)



Epoch: 0



Best accuracy: 47.57

Epoch: 1



Best accuracy: 49.56

Epoch: 2



Best accuracy: 54.56

Epoch: 3



Best accuracy: 57.12

Epoch: 4


...


Best accuracy: 80.37

Epoch: 96



Best accuracy: 80.37

Epoch: 97



Best accuracy: 80.37

Epoch: 98



Best accuracy: 80.38

Epoch: 99



Best accuracy: 80.37
